<a target="_blank" href="https://colab.research.google.com/github/Moikii/SHEL">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [15]:
!git clone https://github.com/Moikii/SHEL.git

fatal: destination path 'SHEL' already exists and is not an empty directory.


In [1]:
%%capture
!pip install ultralytics

In [17]:
import sys
import os
from ultralytics import YOLO

py_file_location = "/content/SHEL/src"
sys.path.append(os.path.abspath(py_file_location))

import dataset_generation_functions as dgf
import photo_preparation_functions as ppf

In [ ]:
create_new_dataset = True
dataset_size = 'small' # 'large'


number_of_images = 10
max_number_of_cards_per_image = 4
min_size = 0.2
max_size = 0.7
max_rotation = 360
overlapping = True
seed = 42

epochs = 2

In [19]:
%%capture

if create_new_dataset:
  backgroundsID = '1KHAYHo9voKRUWtff5ES07hBRUmT7vSg5'
  !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$fileID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$backgroundsID -O backgrounds.zip && rm -rf /tmp/cookies.txt
  !unzip backgrounds.zip -d ./backgrounds
  !rm backgrounds.zip

  if 'small':
    photosID = '1Ot7kAToB1YTSraS3hgbw9zBHDmHtZhpj'
  elif 'large':
    photosID = '1o0f3q5zJzfRXs9vSMb1uR9KcUfWtFItZ'
  else:
    print('small or large are the two options!')
  !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$fileID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$photosID -O photos.zip && rm -rf /tmp/cookies.txt
  !unzip photos.zip -d ./photos
  !rm photos.zip

  PHOTOS_DIR = '/content/photos'
  BACKGROUNDS_DIR = '/content/backgrounds'
  DATASET_DIR = '/content/dataset'

  PLAYING_CARDS_DIR = ppf.process_photos(PHOTOS_DIR)
  DATASET_DIR = dgf.generate_dataset(BACKGROUNDS_DIR, PLAYING_CARDS_DIR, DATASET_DIR, number_of_images, max_number_of_cards_per_image,
                      min_size, max_size, max_rotation, overlapping, seed)

else:
  if 'small':
    datasetID = '1o6z8eRo0pKg9IXQR80-E8p7XvwQMXLwX'
  elif 'large':
    datasetID = '11cMzPUd_ETyOzzb0iJMtONWP10Nu7Zwc'
  else:
    print('small or large are the two options!')

  !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$fileID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$datasetID -O dataset.zip && rm -rf /tmp/cookies.txt
  !unzip dataset.zip -d ./dataset
  !rm dataset.zip

  DATASET_DIR = '/content/dataset'

In [ ]:
model = YOLO('yolov8n.pt')
results = model.train(data = DATASET_DIR + '/data.yaml', epochs = epochs)